# **1. Import Modul**

In [8]:
!pip install lightgbm
!pip install scikit-learn

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.5 MB 1.7 MB/s eta 0:00:01
     ------- -------------------------------- 0.3/1.5 MB 2.8 MB/s eta 0:00:01
     ------------ --------------------------- 0.5/1.5 MB 3.6 MB/s eta 0:00:01
     -------------- ------------------------- 0.5/1.5 MB 2.8 MB/s eta 0:00:01
     ---------------- ----------------------- 0.6/1.5 MB 2.5 MB/s eta 0:00:01
     ----------------- ---------------------- 0.6/1.5 MB 2.4 MB/s eta 0:00:01
     ------------------- -------------------- 0.7/1.5 MB 2.1 MB/s eta 0:00:01
     ------------------- -------------------- 0.7/1.5 MB 2.1 MB/s eta 0:00:01
     ------------------- -------------------- 0.7/1.5 MB 2.1 MB/s eta 0:00:01
     ------------------- -------------------- 0.7/1.5 MB 2.1 MB/s eta 0:00:01
     ------------------- -------------------- 0.7/1.5 MB 2.1 MB/s eta 0:00:01
     ------------------- -------------------- 0.7/1.5 MB 2.1 MB/s eta 0


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import (
    make_scorer, roc_auc_score, roc_auc_score, accuracy_score,
    precision_score, recall_score, f1_score
)

import pickle

# **2. Read Dataset**

In [11]:
train = pd.read_csv("../datasets/train (1).csv")
test = pd.read_csv("../datasets/test (4).csv")
submission = pd.read_csv("../datasets/sample_submission.csv")

print("Train shape:", train.shape)
display(train.head())

print("Test shape:", test.shape)
display(test.head())

print("Submission shape:", submission.shape)
display(submission.head())

Train shape: (1176, 36)


,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,VM684,47,Travel_Rarely,1225,Sales,2,4,Life Sciences,1,1676,...,80,3,29,2,3,3,2,1,2,0
1,ZO618,22,Travel_Rarely,594,Research & Development,2,1,Technical Degree,1,169,...,80,1,3,2,3,2,1,2,1,0
2,JO233,46,Travel_Rarely,406,Sales,3,1,Marketing,1,1124,...,80,1,23,3,3,12,9,4,9,0
3,GW565,25,Travel_Rarely,622,Sales,13,1,Medical,1,645,...,80,0,7,1,3,7,4,0,6,0
4,OZ187,43,Travel_Frequently,1001,Research & Development,9,5,Medical,1,663,...,80,1,10,3,3,8,7,4,7,0


Test shape: (294, 35)


,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,CM617,24,Non-Travel,830,Sales,13,2,Life Sciences,1,1495,...,3,80,1,1,2,3,1,0,0,0
1,PJ010,44,Travel_Rarely,1117,Research & Development,2,1,Life Sciences,1,1246,...,4,80,1,10,5,3,10,5,7,7
2,GJ831,31,Travel_Rarely,688,Sales,7,3,Life Sciences,1,613,...,3,80,1,10,3,2,5,4,0,1
3,JD352,44,Travel_Rarely,1199,Research & Development,4,2,Life Sciences,1,1288,...,4,80,2,26,4,2,25,9,14,13
4,WZ263,36,Travel_Rarely,318,Research & Development,9,3,Medical,1,90,...,1,80,1,2,0,2,1,0,0,0


Submission shape: (294, 2)


,id,Attrition
0,CM617,1
1,PJ010,1
2,GJ831,1
3,JD352,0
4,WZ263,1


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        1176 non-null   object
 1   Age                       1176 non-null   int64 
 2   BusinessTravel            1176 non-null   object
 3   DailyRate                 1176 non-null   int64 
 4   Department                1176 non-null   object
 5   DistanceFromHome          1176 non-null   int64 
 6   Education                 1176 non-null   int64 
 7   EducationField            1176 non-null   object
 8   EmployeeCount             1176 non-null   int64 
 9   EmployeeNumber            1176 non-null   int64 
 10  EnvironmentSatisfaction   1176 non-null   int64 
 11  Gender                    1176 non-null   object
 12  HourlyRate                1176 non-null   int64 
 13  JobInvolvement            1176 non-null   int64 
 14  JobLevel                

# **3. Quick EDA**

In [13]:
print(f"Duplikasi Train: {train.duplicated().sum()}")
print(f"Duplikasi Test: {test.duplicated().sum()}")

Duplikasi Train: 0
Duplikasi Test: 0


In [14]:
train.isna().sum()

id                          0
Age                         0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [15]:
test.isna().sum()

id                          0
Age                         0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [16]:
print(train['EmployeeCount'].value_counts())
print(test['EmployeeCount'].value_counts())

EmployeeCount
1    1176
Name: count, dtype: int64
EmployeeCount
1    294
Name: count, dtype: int64


In [17]:
print(train['EmployeeNumber'].nunique())
print(train.shape)

print(test['EmployeeNumber'].nunique())
print(test.shape)

1176
(1176, 36)
294
(294, 35)


In [18]:
print(train['Over18'].value_counts())
print(test['Over18'].value_counts())

Over18
Y    1176
Name: count, dtype: int64
Over18
Y    294
Name: count, dtype: int64


In [19]:
train.drop(columns=['EmployeeCount', 'EmployeeNumber', 'Over18'], inplace=True)
test.drop(columns=['EmployeeCount', 'EmployeeNumber', 'Over18'], inplace=True)

# **4. Splitting Dataset**

In [20]:
X = train.drop(columns=['id', 'Attrition'])
y = train['Attrition']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((940, 31), (236, 31), (940,), (236,))

# **5. Pre-Processing**

In [23]:
# === Feature Engineering dilakukan sebelum split ===
for df in [train, test]:
    df['TenureRatio'] = df['YearsAtCompany'] / (df['TotalWorkingYears'] + 1)
    df['PromotionRate'] = df['YearsSinceLastPromotion'] / (df['YearsAtCompany'] + 1)
    df['WorkSatisfaction'] = (
        df['JobSatisfaction'] +
        df['EnvironmentSatisfaction'] +
        df['RelationshipSatisfaction']
    ) / 3
    df['IncomePerHour'] = df['MonthlyIncome'] / (df['StandardHours'] + 1)
    df['OverTimeFlag'] = (df['OverTime'] == 'Yes').astype(int)

# === Buat X dan y ===
X = train.drop(columns=['id', 'Attrition'])
y = train['Attrition']

# === Split data ===
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# === Sinkronisasi fitur numerik dan kategorikal ===
numerical_features = [col for col in numerical_features if col in X_train.columns]
categorical_features = [col for col in categorical_features if col in X_train.columns]


In [21]:
"""# **5. Pre-Processing (Developed Version)**"""

# ==== Feature Engineering ====
for df in [train, test]:
    # Rasio lama bekerja terhadap pengalaman total
    df['TenureRatio'] = df['YearsAtCompany'] / (df['TotalWorkingYears'] + 1)
    
    # Rasio promosi terhadap lama bekerja
    df['PromotionRate'] = df['YearsSinceLastPromotion'] / (df['YearsAtCompany'] + 1)
    
    # Rata-rata kepuasan kerja (gabungan 3 variabel)
    df['WorkSatisfaction'] = (
        df['JobSatisfaction'] +
        df['EnvironmentSatisfaction'] +
        df['RelationshipSatisfaction']
    ) / 3
    
    # Rasio penghasilan terhadap jam kerja
    df['IncomePerHour'] = df['MonthlyIncome'] / (df['StandardHours'] + 1)
    
    # Indikator lembur numerik (1 jika Yes)
    df['OverTimeFlag'] = (df['OverTime'] == 'Yes').astype(int)

# ==== Update daftar fitur ====
numerical_features = [
    'Age', 'DailyRate', 'DistanceFromHome', 'HourlyRate', 'MonthlyIncome', 
    'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'StandardHours', 
    'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany', 
    'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
    # fitur baru
    'TenureRatio', 'PromotionRate', 'WorkSatisfaction', 'IncomePerHour', 'OverTimeFlag'
]

categorical_features = [
    'BusinessTravel','Department','Education','EducationField','EnvironmentSatisfaction',
    'Gender','JobInvolvement','JobLevel','JobRole','JobSatisfaction','MaritalStatus',
    'OverTime','PerformanceRating','RelationshipSatisfaction','StockOptionLevel','WorkLifeBalance'
]

# ==== Pipeline numerik dan kategorikal ====
numerical_pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler())  # scaling agar tiap fitur punya rentang seimbang
])

categorical_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # encoding kategori
])

# ==== Gabung semua preprocessing ====
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)


In [ ]:
numerical_features = [
    'Age', 'DailyRate', 'DistanceFromHome',
    'HourlyRate', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
    'PercentSalaryHike', 'StandardHours', 'TotalWorkingYears', 'TrainingTimesLastYear',
    'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion','YearsWithCurrManager'
]

categorical_features = [
    'BusinessTravel','Department','Education', 'EducationField', 'EnvironmentSatisfaction','Gender','JobInvolvement',
    'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus','OverTime', 'PerformanceRating',
    'RelationshipSatisfaction','StockOptionLevel', 'WorkLifeBalance'
]

numerical_pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)

# **6. Training**

In [24]:
"""# **6. Training - Model LightGBM (LGBMClassifier)**"""

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LGBMClassifier(random_state=42))
])

# Parameter grid untuk tuning
param_grid = {
    'model__n_estimators': [300, 500, 700],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__num_leaves': [15, 31, 63],
    'model__max_depth': [-1, 5, 8],
    'model__subsample': [0.8, 1.0],
    'model__colsample_bytree': [0.8, 1.0]
}

# StratifiedKFold Cross Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV dengan AUC sebagai metrik utama
grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=skf,
    n_jobs=-1,
    verbose=2
)

# Fit model ke data train
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Mean AUC:", grid_search.best_score_)


Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[LightGBM] [Info] Number of positive: 152, number of negative: 788
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000846 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1610
[LightGBM] [Info] Number of data points in the train set: 940, number of used features: 82
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.161702 -> initscore=-1.645618
[LightGBM] [Info] Start training from score -1.645618
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

# **7. Evaluation**

In [25]:
best_model = grid_search.best_estimator_

# Prediksi probabilitas & kelas pada data validasi
y_prob = best_model.predict_proba(X_val)[:, 1]
y_pred = best_model.predict(X_val)

# Hitung metrik evaluasi
auc = roc_auc_score(y_val, y_prob)
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print("\n=== LightGBM Model Evaluation ===")
print(f"AUC: {auc:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


=== LightGBM Model Evaluation ===
AUC: 0.7844
Accuracy: 0.8814
Precision: 0.7083
Recall: 0.4474
F1 Score: 0.5484


# **8. Prediction**

In [26]:
# Prediksi probabilitas untuk data test
test_pred_prob = best_model.predict_proba(test)[:, 1]

# Masukkan ke kolom Attrition pada submission
submission['Attrition'] = test_pred_prob

print("\nContoh hasil prediksi probabilitas:")
display(submission.head())

# Simpan hasil prediksi ke file CSV
submission.to_csv('submission_improved.csv', index=False)
print("\nFile 'submission_improved.csv' berhasil dibuat!")


Contoh hasil prediksi probabilitas:


,id,Attrition
0,CM617,5.418286e-04
1,PJ010,2.123100e-08
2,GJ831,1.259056e-04
3,JD352,7.325471e-11
4,WZ263,7.131218e-03



File 'submission_improved.csv' berhasil dibuat!


# **9. Save Model**

In [27]:
# Simpan pipeline terbaik sebagai file .pkl
import pickle
with open("best_model_improved.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("Pipeline LightGBM tersimpan sebagai 'best_model_improved.pkl'")

Pipeline LightGBM tersimpan sebagai 'best_model_improved.pkl'


In [ ]:
# model-model
models = {
    "SVC": SVC(probability=True, random_state=42),  # probability=True untuk AUC
    "RandomForest": RandomForestClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "XGBClassifier": XGBClassifier(use_label_encoder=False,
                                   eval_metric='logloss',
                                   random_state=42),
    "LGBMClassifier": LGBMClassifier(random_state=42, verbose=-1)  # hilangkan log
}

# scorer AUC
auc_scorer = make_scorer(roc_auc_score, needs_proba=True)

In [ ]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for name, model in models.items():
    print(f"\n=== {name} ===")
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    fold = 1
    auc_scores = []
    for train_idx, test_idx in skf.split(X_train, y_train):
        # tampilkan index split
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[test_idx]

        # fit dan prediksi prob
        pipe.fit(X_tr, y_tr)
        y_prob = pipe.predict_proba(X_val)[:,1]

        auc = roc_auc_score(y_val, y_prob)
        print(f"Fold {fold} - AUC: {auc:.4f}")
        auc_scores.append(auc)
        fold += 1

    print(f"{name} - Mean AUC: {sum(auc_scores)/len(auc_scores):.4f}")



=== SVC ===
Fold 1 - AUC: 0.7968
Fold 2 - AUC: 0.7910
Fold 3 - AUC: 0.8113
SVC - Mean AUC: 0.7997

=== RandomForest ===
Fold 1 - AUC: 0.7820
Fold 2 - AUC: 0.7676
Fold 3 - AUC: 0.8153
RandomForest - Mean AUC: 0.7883

=== LogisticRegression ===
Fold 1 - AUC: 0.8233
Fold 2 - AUC: 0.8345
Fold 3 - AUC: 0.8661
LogisticRegression - Mean AUC: 0.8413

=== XGBClassifier ===
Fold 1 - AUC: 0.7977
Fold 2 - AUC: 0.7864
Fold 3 - AUC: 0.7907
XGBClassifier - Mean AUC: 0.7916

=== LGBMClassifier ===
Fold 1 - AUC: 0.7944
Fold 2 - AUC: 0.7682
Fold 3 - AUC: 0.8245
LGBMClassifier - Mean AUC: 0.7957


Logistic Regression

In [ ]:
# ============================================
# 6. Training - Menggunakan XGBoost Classifier
# ============================================

# Pipeline dengan preprocessor dan XGBoost
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    ))
])

# Parameter grid untuk tuning
param_grid = {
    'model__n_estimators': [200, 400, 600],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__max_depth': [3, 5, 7],
    'model__subsample': [0.8, 1.0],
    'model__colsample_bytree': [0.8, 1.0]
}

# Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV dengan AUC sebagai metrik utama
grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=skf,
    n_jobs=-1,
    verbose=2
)

# Fit model ke data train
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Mean AUC:", grid_search.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'model__colsample_bytree': 0.8, 'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 400, 'model__subsample': 0.8}
Best Mean AUC: 0.8213210036767032


# **7. Hyperparameter Tuning**

In [ ]:
# pipeline
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(max_iter=1000, random_state=42))
])

# parameter grid untuk tuning
param_grid = {
    'model__C': [0.01, 0.1, 1, 10, 100],       # regularisasi
    'model__penalty': ['l1', 'l2'],            # jenis regularisasi
    'model__solver': ['liblinear']             # solver yang mendukung l1/l2
}

# StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV dengan AUC sebagai scoring
grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=skf,
    n_jobs=-1,
    verbose=1
)

# fit
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['Age',
                                                                          'DailyRate',
                                                                          'DistanceFromHome',
                                                                          'HourlyRate',
                                                                          'MonthlyIncome',
                                                                          'MonthlyRate',
                                                                          'NumCompaniesWorked',
                                                                          'PercentSalaryHike',
                                                                          'StandardHours',
                                                                          'TotalWorkingY...
                                                                          'JobLevel',
                                                                          'JobRole',
                                                                          'JobSatisfaction',
                                                                          'MaritalStatus',
                                                                          'OverTime',
                                                                          'PerformanceRating',
                                                                          'RelationshipSatisfaction',
                                                                          'StockOptionLevel',
                                                                          'WorkLifeBalance'])])),
                                       ('model',
                                        LogisticRegression(max_iter=1000,
                                                           random_state=42))]),
             n_jobs=-1,
             param_grid={'model__C': [0.01, 0.1, 1, 10, 100],
                         'model__penalty': ['l1', 'l2'],
                         'model__solver': ['liblinear']},
             scoring='roc_auc', verbose=1)

In [ ]:
print("Best Parameters:", grid_search.best_params_)
print("Best Mean AUC:", grid_search.best_score_)

Best Parameters: {'model__C': 1, 'model__penalty': 'l1', 'model__solver': 'liblinear'}
Best Mean AUC: 0.846464565024591


In [ ]:
# =================================================
# 7. Evaluation - Mengukur performa model terbaik
# =================================================

best_model = grid_search.best_estimator_

# Prediksi probabilitas & kelas pada data validasi
y_prob = best_model.predict_proba(X_val)[:, 1]
y_pred = best_model.predict(X_val)

# Hitung metrik evaluasi
auc = roc_auc_score(y_val, y_prob)
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print("\n=== XGBoost Model Evaluation ===")
print(f"AUC: {auc:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")



=== XGBoost Model Evaluation ===
AUC: 0.9291
Accuracy: 0.9265
Precision: 0.9667
Recall: 0.5686
F1 Score: 0.7160


# **8. Evaluation**

In [ ]:
# Ambil model terbaik dari grid search
best_model = grid_search.best_estimator_

# Prediksi probabilitas untuk AUC
y_prob = best_model.predict_proba(X_val)[:,1]

# Prediksi kelas
y_pred = best_model.predict(X_val)

# Evaluasi metrik
auc = roc_auc_score(y_val, y_prob)
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print("=== Logistic Regression Evaluation ===")
print(f"AUC: {auc:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

=== Logistic Regression Evaluation ===
AUC: 0.9291
Accuracy: 0.9265
Precision: 0.9667
Recall: 0.5686
F1 Score: 0.7160


In [ ]:
# ==========================================================
# 8. Prediction - Prediksi data test untuk file submission
# ==========================================================

test_pred_prob = best_model.predict_proba(test)[:, 1]
submission['Attrition'] = test_pred_prob

print("\nContoh hasil prediksi probabilitas:")
display(submission.head())

# Simpan hasil prediksi ke file CSV
submission.to_csv('submission_xgb.csv', index=False)
print("\nFile 'submission_xgb.csv' berhasil dibuat!")



Contoh hasil prediksi probabilitas:


,id,Attrition
0,CM617,0.069426
1,PJ010,0.002633
2,GJ831,0.108429
3,JD352,0.006221
4,WZ263,0.309898



File 'submission_xgb.csv' berhasil dibuat!


# **9. Prediction**

In [ ]:
test_pred = best_model.predict(test)

In [ ]:
test_pred_prob = best_model.predict_proba(test)[:,1]

In [ ]:
submission['Attrition'] = test_pred_prob

In [ ]:
submission.head()

,id,Attrition
0,CM617,0.069426
1,PJ010,0.002633
2,GJ831,0.108429
3,JD352,0.006221
4,WZ263,0.309898


In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
import os

# Create the directory if it doesn't exist
output_dir = '../datasets'
os.makedirs(output_dir, exist_ok=True)

# **10. Save Model**

In [ ]:
# best_model adalah pipeline hasil GridSearchCV
best_pipeline = grid_search.best_estimator_

# simpan pipeline ke file .pkl
with open("best_logistic_pipeline.pkl", "wb") as f:
    pickle.dump(best_pipeline, f)

print("Pipeline tersimpan sebagai 'best_logistic_pipeline.pkl'")

Pipeline tersimpan sebagai 'best_logistic_pipeline.pkl'


In [ ]:
# # -----------------------
# # untuk load kembali
# with open("best_logistic_pipeline.pkl", "rb") as f:
#     loaded_pipeline = pickle.load(f)

# # prediksi ke data baru
# y_pred_new = loaded_pipeline.predict(X_new)
# y_prob_new = loaded_pipeline.predict_proba(X_new)[:,1]

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import GridSearchCV

# Base models
estimators = [
    ('rf', RandomForestClassifier(n_estimators=200, random_state=42)),
    ('xgb', XGBClassifier(
        use_label_encoder=False, eval_metric='logloss',
        random_state=42, n_estimators=300, learning_rate=0.05, max_depth=5
    )),
    ('lgbm', LGBMClassifier(
        random_state=42, n_estimators=300, learning_rate=0.05, num_leaves=31
    ))
]

# Meta-model (Logistic Regression)
stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(max_iter=1000, random_state=42),
    cv=5,
    n_jobs=-1
)

pipe_stack = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', stack_model)
])

# Cross-validation untuk evaluasi AUC
cv_auc = cross_val_score(pipe_stack, X_train, y_train, cv=5,
                         scoring='roc_auc', n_jobs=-1)
print("Mean CV AUC:", np.mean(cv_auc))

# Fit final model
pipe_stack.fit(X_train, y_train)

# Prediksi probabilitas
y_prob = pipe_stack.predict_proba(X_val)[:,1]
auc = roc_auc_score(y_val, y_prob)
print(f"AUC Validation: {auc:.4f}")


Mean CV AUC: 0.801546850874615
AUC Validation: 1.0000


In [ ]:
# =====================================================
# 9. Save Model - Simpan pipeline terbaik ke file .pkl
# =====================================================

import pickle
with open("best_xgb_pipeline.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("Pipeline XGBoost tersimpan sebagai 'best_xgb_pipeline.pkl'")

# # Untuk load kembali:
# with open("best_xgb_pipeline.pkl", "rb") as f:
#     loaded_pipeline = pickle.load(f)
# y_pred_new = loaded_pipeline.predict(X_new)
# y_prob_new = loaded_pipeline.predict_proba(X_new)[:,1]

Pipeline XGBoost tersimpan sebagai 'best_xgb_pipeline.pkl'


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 8.5 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': 42,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }
    model = XGBClassifier(**params)
    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    score = cross_val_score(pipe, X_train, y_train, cv=3, scoring='roc_auc', n_jobs=-1)
    return np.mean(score)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25)
print("Best params:", study.best_params)
print("Best AUC:", study.best_value)


[I 2025-10-26 08:43:05,741] A new study created in memory with name: no-name-0478582a-9857-4af6-9891-0f68e9087b68
[I 2025-10-26 08:43:06,015] Trial 0 finished with value: 0.811880988194773 and parameters: {'n_estimators': 496, 'max_depth': 4, 'learning_rate': 0.17412626265822093, 'subsample': 0.7501214836205173, 'colsample_bytree': 0.746850121487348, 'gamma': 2.1116681853511836}. Best is trial 0 with value: 0.811880988194773.
[I 2025-10-26 08:43:06,381] Trial 1 finished with value: 0.81049977908713 and parameters: {'n_estimators': 761, 'max_depth': 10, 'learning_rate': 0.12199034191825413, 'subsample': 0.7867266767202716, 'colsample_bytree': 0.6389742572617757, 'gamma': 1.330228596456049}. Best is trial 0 with value: 0.811880988194773.
[I 2025-10-26 08:43:06,848] Trial 2 finished with value: 0.8100645133874718 and parameters: {'n_estimators': 974, 'max_depth': 10, 'learning_rate': 0.24689165091849444, 'subsample': 0.6030770535925738, 'colsample_bytree': 0.9400016157350886, 'gamma': 4.3

Best params: {'n_estimators': 666, 'max_depth': 3, 'learning_rate': 0.023405772946534627, 'subsample': 0.697804530259148, 'colsample_bytree': 0.8243668267477967, 'gamma': 2.2736975497180953}
Best AUC: 0.816372897130485


In [ ]:
X['YearsAtCompany_per_JobLevel'] = X['YearsAtCompany'] / (X['JobLevel'] + 1)
X['Income_per_Experience'] = X['MonthlyIncome'] / (X['TotalWorkingYears'] + 1)
X['PromotionRate'] = X['YearsSinceLastPromotion'] / (X['YearsAtCompany'] + 1)


In [ ]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(best_model, X, y, cv=cv, scoring='roc_auc')
print("Mean CV AUC:", scores.mean())


Mean CV AUC: 0.8549430936648982


In [ ]:
final_model = pipe_stack  # atau hasil Optuna
final_model.fit(X, y)
submission['Attrition'] = final_model.predict_proba(test)[:,1]
submission.to_csv('submission_improved.csv', index=False)
